# PAMD computer lab week 9

In this week's lab, we will implement regression and classification trees, as well as ensemble methods for them (random forests).

## Section 1: Classification trees

We have discussed in the lecture how decision trees can be used for both classification and regression tasks. In this first example, we will build one for the classification of a binary outcome - churn.

Churn refers to whether a customer turns away from a company. It's an important measure that companies are interested in both assessing and predicting. For example, they might want to target customers who they think might "leave" with additional advertisting to keep them, or they might want to stop spending money on them early because they don't want to "waste" expenses on a customer who will leave anyway. In real life, you will often not only classify customers by their churn likelihood, but co-analyse churn in combination with profitability.

I have added a churn dataset to Learn. Please download it and upload it to your Noteable space.

### 1.1 Loading in and understanding data

In [2]:
import pandas as pd
import numpy as np

# Import your dataset and plot a few lines of it

df = pd.read_csv("churn_ibm.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


We see various variables, both demographic ones (Partner (Yes/No), tenure (length of contract), gender, etc.), as well as service-related variables (DeviceProtection, PaymentMethod, Contact, MonthlyCharges, etc.). All variables are relatively self-explanatory.

Our outcome variable "Churn" is in the last column. You might want to start with some descriptive statistics to get a feel for the data. How many customers will turn away from the company? What is the overall split between different demographic categories? What's the distribution of their charges (profitability?)?

### 1.2 Data pre-processing

Let's split the data into independent (X) and dependent (y) parts.

Customer ID is just an identifier and can not be used to predict anything, so we drop it.

Something else we should consider is that the decision tree implementation in Python which we're going to use can only deal with numeric variables (i.e. not "strings"). You will remember from the lecture that we talked about how trees can handle different data types well in theory, but for implementation purposes we use dummy variables here.

This allows the tree to do its binary splitting.

Checking data types is good practice in general - it can also highlight problems that you might encounter with your data, for example if a variable has been loaded in incorrectly. 

The dependent variable is also categorical. This is fine because we are predicting a categorical outcome with out classification tree versus a regression tree which would need a numerical outcome. We convert it to a dummy variable here to get a 0/1 outcome.


**TASK** 
- Split your data into X and y.
- Use the 'dtypes' function on X to check the data types of your variables
- Convert categorical ('object') variables in both X and y into dummy variables using the get_dummies function in pandas (documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html))

In [28]:
y = df["Churn"]
X = df.drop(["Churn", "customerID"], axis=1)

X = pd.get_dummies(X)
y = pd.get_dummies(y)

X

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,29.85,29.85,True,False,False,True,True,False,...,False,True,False,False,False,True,False,False,True,False
1,0,34,56.95,1889.50,False,True,True,False,True,False,...,False,False,True,False,True,False,False,False,False,True
2,0,2,53.85,108.15,False,True,True,False,True,False,...,False,True,False,False,False,True,False,False,False,True
3,0,45,42.30,1840.75,False,True,True,False,True,False,...,False,False,True,False,True,False,True,False,False,False
4,0,2,70.70,151.65,True,False,True,False,True,False,...,False,True,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,24,84.80,1990.50,False,True,False,True,False,True,...,True,False,True,False,False,True,False,False,False,True
7028,0,72,103.20,7362.90,True,False,False,True,False,True,...,True,False,True,False,False,True,False,True,False,False
7029,0,11,29.60,346.45,True,False,False,True,False,True,...,False,True,False,False,False,True,False,False,True,False
7030,1,4,74.40,306.60,False,True,False,True,True,False,...,False,True,False,False,False,True,False,False,False,True


In [29]:
y

,No,Yes
0,True,False
1,True,False
2,False,True
3,True,False
4,False,True
...,...,...
7027,True,False
7028,True,False
7029,True,False
7030,False,True


### 1.3 Building classification trees and making predictions

We can very easily construct our decision tree without changing any parameters using a training and test set as usual. In the lecture we talked about how there is no need to scale the variables (which would be impossible with categorical ones anyway), as the variables are considered separately in the branching.

This process will look very familiar as it is similar to our logistic regression implementation.

**TASKS**
- Split your data into training and test data
- Build a classification tree using the DecisionTreeClassifier() function in scitkit learn (Documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html))
- Calculate the accuracy of the classifier using the accuracy_score() and the roc_auc_score() function in scikit learn metrics (documentation [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)). Make sure to calculate the accuracy on the *test* data.

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)


# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Calculate the ROC AUC score of the classifier
roc_auc = roc_auc_score(y_test, predictions)
print("ROC AUC Score:", roc_auc)

Accuracy: 0.7308056872037915
ROC AUC Score: 0.658043833308231


The default setting for the splitting criterion in DecisionTreeClassifier() is Gini index. You can also try building the model using entropy and comparing the results.

If you want to find out more about your tree, such as the size, you can use the node_count argument.

We can also plot the AUC curve as a measure of performance. Remember that AUC is a comparative measure. We can use it to compare between models. You'll want it to be further away from the a 45 degree line, which is to say, better than random.

**OPTIONAL TASK**

Plot the AUC curve of the above tree using the roc_curve function in [scitkit learn metrics](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve).

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# 示例：假设 X_train, X_test, y_train, y_test 已经被定义并准备好了

# 创建并训练决策树分类器
dtc = DecisionTreeClassifier(criterion="entropy")
dtc.fit(X_train, y_train)

# 使用模型对测试集进行预测
predictions = dtc.predict(X_test)

# 计算分类器的准确度
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# 使用预测概率计算ROC AUC分数
# 注意：这里使用的是预测概率的第二列，因为我们关注的是正类的概率
probs = dtc.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, probs)
print("ROC AUC Score:", roc_auc)

# 计算ROC曲线
fpr, tpr, thresholds = roc_curve(y_test, probs)

# 计算AUC
roc_auc = auc(fpr, tpr)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr, tpr, color="darkorange", lw=2, label="ROC curve (area = %0.2f)" % roc_auc)
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")
plt.show()

Accuracy: 0.7336492890995261


TypeError: list indices must be integers or slices, not tuple

You can also experiment with settings in the tree.

For example, you can adjust the tree based on some things that we talked about in the lecture. Try building a tree with a max depth of 3 and a minimum of 10 samples per leaf. This time, we also use entropy/information gain instead of Gini index.

**TASK**

Build a new DecisionTreeClassifier() with the specifications above and compare the resulting accuracy with the previous model.

In [6]:
# YOUR CODE HERE

You will notice that both our accuracy and AUC are up, possibly due to less overfitting with having a smaller tree with less leaves. You will remember from the lecture that decision trees are prone to overfitting, so this is not surprising.

### 1.3 Visualisaing the decision tree

Visualising decision trees is one of their big advantages, so we'll try that next. The first tree was too big, so we're plotting the second one for which we improved complexity.

There's two main ways of plotting decision trees in scikit learn:

- the quick way using the [plot tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html) function
- the slightly prettier way using the [export_graphviz](https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html) function in scikit learn, which relies on the GraphViz package

**TASK**

Pick one of those two and try it out yourself below. The solution will have both options, so you can check out the other way later.

In [7]:
# YOUR CODE HERE

The value list gives us the number of each class present in each of the leaves. We can see that most leaves predict that churn_yes=0 which means that no churn occurs.

Have a look at the different variables used for the splitting.

Unfortunately, scikit-learn does not offer any pruning capabilities (yet). So, the only way to force simplicity is by working with the tree depth as a parameter. 

# Section 2: Regression trees

Regression trees can be built very much the same as classification trees, so this exercise will be shorter.

We will be using the [diabetes dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset) in sklearn.datasets, as it offers a good mix of variables. We will predict the variable 'progression' which is a quantitative measure of disease progression in patients after one year.

### 2.1 Loading data 

In [8]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes

dataset = load_diabetes()

print(dataset['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [9]:
X = pd.DataFrame(data=dataset['data'],columns=dataset['feature_names'])
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [10]:
y = pd.DataFrame(data=dataset['target'],columns=['progression'])
y.head()

,progression
0,151.0
1,75.0
2,141.0
3,206.0
4,135.0


### 2.2 Building a regression decision tree and making predictions

The code for building this tree is very similar to the one above, just using the [DecisionTreeRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) function instead.

There is one key difference:
As we are predicting a continuous outcome, we can now use error measures as accuracy measures. This gives us the deviation of our predicted value from the observed value. In class we discussed using RSS, which is a closely related error measure. RSS is the absolute sum of the squared errors, MSE is the mean of that value, and RMSE is the root square of MSE. Choosing one over the other is a matter of preference.

**TASK**

- Build a regression tree using the DecisionTreeRegressor() function
- Using RMSE as an accuracy measure, calculate how well the tree is performing

In [11]:
# YOUR CODE HERE

**OPTIONAL TASK**

As before, you can use the max_depth and the min_samples_leaf parameters to restrict how deep the tree should be. Try that out yourself and check how the performance of the tree is changing.

In [12]:
# YOUR CODE HERE

### 2.3 Visualising the regression tree

As before, you can visualise the tree to understand better how the predictions were made. Remember that the value entry gives us the -average- of the dependent variable over all the samples in the leaf node.

Notice how some variables are chosen multiple times for splits, on the right side you will even see that BMI is chosen twice directly after itself. This is due to the greedy top down approach that we are using. We are not considering the past or future in our splits - we're just doing the best possible split in a given moment.

**OPTIONAL TASK**

Using the same functions as in 1.3, visualise the smaller tree you just built.

In [13]:
# YOUR CODE HERE

# Section 3: Ensemble learning (random forests, boosting)

You have now built two basic decision trees.

In the lecture, we talked about the disadvantages of decision trees and how they are sometimes not very accurate in their predictions. There are a couple of ways of overcoming that.

Inaccuracy through overfitting can be reduced by pruning. Another way of improving accuracy is by using multiple trees at once - an ensemble of them. Ensemble learning is a whole area in machine learning which aims to improve the accuracy of a model by combining multiple of them.

Scikit learn has a dedicated collection of functions for different ensemble models [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble).

In this activity we will build a random forest and a boosting model for the churn dataset. 

Let's first import the data.

### 3.1 Loading the data

We use our churn dataset from Section 1 again.

In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('churn_ibm.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


You already know the pre-processing steps from earlier:

In [15]:
y = df['Churn']
X = df.drop(['Churn','customerID'],axis=1)

for column in X.columns:
    if X[column].dtype == object:
        X = pd.concat([X,pd.get_dummies(X[column], prefix=column, drop_first=True)],axis=1).drop([column],axis=1)
        
y = pd.get_dummies(y, prefix='churn', drop_first=True)

### 3.2 Random Forests

Remember random forests: they are ensembles of decision trees that are forced to use randomized predictors in their splits. They then all vote together on an outcome.

Scikit learn allows us to construct a random forest quite easily using the RandomForestClassifier() function. Have a look at the documentation first [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

**TASK**
- Split your dataset into training/test
- Using the RandomForestClassifier() function, build a random forest model to predict customer churn on the dataset above. You can choose any number of trees, I will use 20.
- Assess the accuracy of the model using the accuracy_score() function from scikit learn

In [16]:
# YOUR CODE HERE

There's a lot more to play around with in the RandomForestClassifier function. You can print the instance of RandomForestClassifier() to learn about the default settings used, and if you're wondering which settings your model used print get_params() for it.

This can be very important when reporting your results. For example, we can see that bootstrapping is defaulted to TRUE.

**OPTIONAL TASK**

Get to know your random forest model and try out different parameter settings. For example, try increasing your number of trees and compare the accuracy.

In [17]:
# YOUR CODE HERE

### 3.3 Boosted trees

Boosting is a way of learning from past behavior. We are building a sequence of small trees, each of which will use the errors that the previous one did to improve its own performance.

sklearn allows us to implement two versions of the boosting method: AdaBoost and Gradient Boosting. Let's implement AdaBoost as an example, with the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier).

**TASK**
- You can use the test/training data from earlier
- Implement the AdaBoostClassifier() from sklearn.ensemble for our churn data
- Compare the accuracy and AUC of the model to the random forest from earlier

In [18]:
# YOUR CODE HERE

**OPTIONAL TASK**

Find out how AdaBoost is currently performing and try improving that. You can find the number of currently used trees using the  get_params() argument. What happens when you increase the number of trees?

In [19]:
# YOUR CODE HERE

For me, increasing the number of trees improved neither accuracy, nor AUC are drastically, but AdaBoost performs better than the Random Forest we built.

What does that tell us about our data? There might be observations which are particularly difficult to pinpoint for the tree(s). Booster trees can perform well in such situations by focusing and weighting those mistakes heavier.

### 3.4 Feature importance

For all our models, we can calculate the feature importance. This is the (average) reduction in Gini impurity across all trees.

Feature importance can be an important way of interpreting your results. It's also very interesting to compare this across different methods and understand how different algorithms value different features.


**DEMO**

I will demonstrate how to calculate feature importance for my tree below. You can try this out yourself for different trees and what happens if you change the parameters of your models.

In [20]:
# Random forest - 5 most important features

rf = RandomForestClassifier(n_estimators=20) # build RF with 20 trees
rf.fit(X_train,y_train.values.ravel()) # fit it to training data

for c, column in enumerate(X_test.columns):
    if rf.feature_importances_[c] in sorted(rf.feature_importances_)[-5:]:
        print('Variable',column,rf.feature_importances_[c])

NameError: name 'RandomForestClassifier' is not defined

In [ ]:
# AdaBoost - 5 most important features

ada = AdaBoostClassifier()
ada.fit(X_train,y_train.values.ravel())

for c, column in enumerate(X_test.columns):
    if ada.feature_importances_[c] in sorted(ada.feature_importances_)[-5:]:
        print('Variable',column,ada.feature_importances_[c])

Variable tenure 0.16
Variable MonthlyCharges 0.26
Variable TotalCharges 0.26
Variable InternetService_Fiber optic 0.06
Variable Contract_Two year 0.04


It seems that both random forests and AdaBoost ahave exactly the same variables driving their Gini impurity down. Mostly the length of tenure, monthly charges, total charges, being connected through fiber and the contract length.

Having both methods value the same features can strengthen our recommendations made to the company.